In [30]:
import os 
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold, content_types
from collections.abc import Iterable
from function_calls import final_factuality_factor_score, emotion_analyzer
from google.protobuf.struct_pb2 import Struct


In [31]:
api_key = os.getenv("API_KEY")
genai.configure(api_key=api_key)

Documentation: https://github.com/google/generative-ai-docs/blob/main/site/en/gemini-api/docs/function-calling/python.ipynb

In [32]:
# tools = {
#   "functions": [
#       {
#         "name": "emotion_analyzer",
#         "description": "Analyzes the emotionality and exaggeration level in a given text and returns a scaled score.",
#         "parameters": {
#           "type": "object",
#           "properties": {
#             "text": {
#               "type": "string",
#               "description": "The text content to analyze for emotional intensity and exaggeration."
#             }
#           },
#           "required": ["text"]
#         }
#       },
#       {
#         "name": "final_factuality_factor_score",
#         "description": "Averages the microfactors from a single factuality factor. This function should be used when combining into an overall score.",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "microfactor_1": {
#                     "type": "float",
#                     "description": "First microfactor for a factuality factor, used to help calculate the factuality factor"
#                 },
#                 "microfactor_2": {
#                     "type": "float",
#                     "description": "Second microfactor for a factuality factor, used to help calculate the factuality factor"
#                 },
#                 "microfactor_3": {
#                     "type": "float",
#                     "description": "Third microfactor for a factuality factor, used to help calculate the factuality factor"
#                 }
#             },
#             "required": ["microfactor_1", "microfactor_2", "microfactor_3"]
#         }
#       }
#   ]
# }


In [33]:
generation_config = {
    "max_output_tokens": 4000, # less output, means faster
    "response_mime_type": "text/plain",
    "temperature": 1, # higher temp --> more risks the model takes with choices
    "top_p": 0.95, # how many tokens are considered when producing outputs
    "top_k": 40, # token is selected from 40 likely tokens
}

safety_settings = {
  HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
  HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
}

def tool_config_from_mode(mode: str, fns: Iterable[str] = ()):
    """Create a tool config with the specified function calling mode."""
    return content_types.to_tool_config(
        {"function_calling_config": {"mode": mode, "allowed_function_names": fns}}
    )

functions = {"final_factuality_factor_score": final_factuality_factor_score,
            "emotion_analyzer": emotion_analyzer,}

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-002",
    generation_config=generation_config,
    safety_settings=safety_settings,
    tools = functions.values(),
    system_instruction="""
        You are trying to fight against misinformation by scoring different articles on their factuality factors. 
        In your responses:
        - Use each function only once per request.
        - Integrate the results from the function calls to generate a complete response.
        - Do not assess an article until you are given a factuality factor to grade on.
        - Be concise and avoid redundant function calls.    
    """
)

In [34]:
# chat = model.start_chat(enable_automatic_function_calling=True)

In [35]:
# response = chat.send_message("""
# "Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!."
# Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:
# 1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.
# 2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.
# 3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.
# Then, combine them into an overall score for sensationalism and explain your thought process.
# Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.
# """)
# print(f"Model response: {response.text}")


In [36]:
def call_function(function_call, functions):
    function_name = function_call.name
    function_args = function_call.args
    return functions[function_name](**function_args)

In [37]:
tool_config = tool_config_from_mode("auto")

response = model.generate_content(
    "'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
Then, combine them into an overall score for sensationalism and explain your thought process.\
Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once.", tool_config=tool_config)
response.candidates[0].content.parts
    

[text: "Here\'s an analysis of the sentence based on the sensationalism factor:\n\n1. **Sensationalism Detection:** The phrase \"Shocking Discovery\" and \"Change Humanity Forever\" are clear indicators of sensationalism. It uses hyperbole to grab attention and create a sense of urgency.  I would score this a 9/10.\n\n2. **Emotion Analysis:** Let\'s analyze the sentence\'s emotionality using the `emotion_analyzer` function:\n\n"
, function_call {
  name: "emotion_analyzer"
  args {
    fields {
      key: "text"
      value {
        string_value: "Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!."
      }
    }
  }
}
, text: "\n\nI\'ll incorporate the result of this function call once you provide it.\n\n3. **Linguistic Database Comparison:**  This requires comparing the sentence\'s phrasing and vocabulary against known patterns in reliable and unreliable sources. Since I don\'t have access to such a database, I\'ll make an educated guess. The exaggerat

In [ ]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "Here's an analysis of the sentence based on the sensationalism factor:\n\n1. **Sensationalism Detection:** The phrase \"Shocking Discovery\" and \"Change Humanity Forever\" are clear indicators of sensationalism. It uses hyperbole to grab attention and create a sense of urgency.  I would score this a 9/10.\n\n2. **Emotion Analysis:** Let's analyze the sentence's emotionality using the `emotion_analyzer` function:\n\n"
              },
              {
                "function_call": {
                  "name": "emotion_analyzer",
                  "args": {
                    "text": "Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!."
                  }
                }
              },
              {
                "tex

In [ ]:
# change the index after parts to update the function call
response.candidates[0].content.parts[3].function_call

name: "final_factuality_factor_score"
args {
  fields {
    key: "microfactor_3"
    value {
      number_value: 8
    }
  }
  fields {
    key: "microfactor_2"
    value {
      number_value: 8
    }
  }
  fields {
    key: "microfactor_1"
    value {
      number_value: 9
    }
  }
}

In [39]:
part = response.candidates[0].content.parts[0]

if part.function_call:
    result = call_function(part.function_call, functions)

print(result)

5.17


In [ ]:
part2 = response.candidates[1].content.parts[0]

In [40]:
s = Struct()
s.update({"result": result})

function_response = genai.protos.Part(
    function_response=genai.protos.FunctionResponse(name=response.candidates[0].content.parts[0].function_call.name, response=s)
)


In [41]:
messages = [
    # fmt: off
    {"role": "user",
     "parts": ["'Shocking Discovery: Scientists Unveil a Secret That Will Change Humanity Forever!.'\
Given the previous sentence, rate the sentence based on the defined microfactors of the factuality factor, sensationalism, from 1 to 10, ensuring that you explain the reasoning behind each score, and feel free to use function calling:\
1. Sensationalism Detection: Identify instancesS of sensationalism in titles and main content.\
2. Emotion Analysis: Assess the writing style for excessive emotionality or exaggeration.\
3. Linguistic Database Comparison: Match linguistic features against databases of both trusted and untrusted sources to ascertain reliability.\
Then, combine them into an overall score for sensationalism and explain your thought process.\
Lastly, please phrase the overall score as 'Normal Prompting Overall sensationalism: {score}', where score is calculated via the final_factuality_factor_score function. Each function call should be used once."]},
    {"role": "model",
     "parts": response.candidates[0].content.parts},
    {"role": "user",
     "parts": [function_response]},
    # fmt: on
]


In [42]:
response = model.generate_content(messages)

InvalidArgument: 400 * GenerateContentRequest.contents[2].parts[0].function_response.name: Name cannot be empty.


In [ ]:
response._result

candidates {
  content {
    parts {
      text: "1. **Sensationalism Detection:** The title uses strong emotional language (\"Shocking,\" \"Secret,\" \"Change Humanity Forever\") and lacks specific details, suggesting a high degree of sensationalism. **Score: 9/10**\n\n2. **Emotion Analysis:** The title evokes strong emotions like surprise and excitement but does not contain factual information.  The provided `emotion_analyzer` function output further supports a moderate level of emotionality. **Score: 5.17/10**\n\n3. **Linguistic Database Comparison:** While I lack access to external databases to perform a direct comparison, the linguistic features (hyperbole, grand claims) are commonly associated with less trustworthy sources.  **Score: 8/10**\n\nTo calculate the overall sensationalism score, I\'ll use the provided function:\n\n"
    }
    parts {
      function_call {
        name: "final_factuality_factor_score"
        args {
          fields {
            key: "microfactor_3"
  